In [1]:
#| default_exp meiosis

In [2]:
#| export

from fastcore.test import test_eq, test_close, test_fail
import jax
import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass

from typing import List, Optional
from jaxtyping import Array, Float, Int, PyTree

from chewc.structs import Population, SimParam
from chewc.popgen import quick_haplo

In [3]:
#| export
# chewc/meiosis.py
from functools import partial
from typing import Tuple

import jax
import jax.numpy as jnp
from jax import lax, vmap

# from chewc.config import SimConfig
from chewc.structs import SimParam, SimConfig, SimState
# from chewc.state import SimState

# ==============================================================================
# JIT-Compiled Meiosis Kernels
# ==============================================================================


@partial(jax.jit, static_argnames=("max_crossovers",))
def _sample_chiasmata(
    key: jax.random.PRNGKey,
    map_length: float,
    v: float,
    max_crossovers: int = 20,
) -> jnp.ndarray:
    """
    Generates crossover positions along a single chromosome using a Gamma process.
    This function is JIT-compatible and remains unchanged from your previous version.
    """
    shape = v
    scale = 1.0 / (2.0 * v)

    def scan_body(carry, _):
        key, last_pos = carry
        key, subkey = jax.random.split(key)
        distance = jax.random.gamma(subkey, shape) * scale
        new_pos = last_pos + distance
        return (key, new_pos), new_pos

    key, initial_key = jax.random.split(key)
    # Start sampling from a random negative position to ensure the first
    # crossover has a chance to be close to the start of the chromosome.
    initial_start_pos = jax.random.uniform(initial_key, minval=-10.0, maxval=0.0)
    init_carry = (key, initial_start_pos)

    _, crossover_positions = lax.scan(
        scan_body, init_carry, None, length=max_crossovers
    )

    # A crossover is only valid if it's within the bounds (0, map_length).
    valid_mask = (crossover_positions > 0) & (crossover_positions < map_length)
    valid_crossovers = jnp.where(valid_mask, crossover_positions, jnp.nan)

    return valid_crossovers


@partial(jax.jit, static_argnames=("max_crossovers",))
def _create_gamete(
    key: jax.random.PRNGKey,
    parental_geno: jnp.ndarray,  # Shape: (ploidy, n_loci)
    parental_ibd: jnp.ndarray,  # Shape: (ploidy, n_loci)
    gen_map: jnp.ndarray,  # Shape: (n_loci,)
    v_interference: float,
    max_crossovers: int,
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Creates a single recombinant gamete (geno and IBD) for one chromosome.

    This kernel is vectorized and JIT-compatible. It now simultaneously
    processes both genotype and IBD information.
    """
    key, chiasma_key, hap_key = jax.random.split(key, 3)

    map_length = gen_map[-1]

    crossover_positions = _sample_chiasmata(
        chiasma_key, map_length, v_interference, max_crossovers
    )

    # Sort the valid crossover positions to handle out-of-order generation
    # from the Gamma process. We fill NaNs with infinity to push them to the end.
    sorted_crossovers = jnp.sort(jnp.nan_to_num(crossover_positions, nan=jnp.inf))

    # `searchsorted` counts how many crossovers occurred before each locus.
    # This creates an array of segment indices.
    n_loci = gen_map.shape[0]
    locus_segments = jnp.searchsorted(
        sorted_crossovers, jnp.arange(n_loci), side="right"
    )

    # Randomly choose which of the two parental haplotypes to start with.
    start_hap = jax.random.choice(hap_key, jnp.array([0, 1], dtype=jnp.uint8))

    # The choice of haplotype (0 or 1) alternates at each crossover.
    haplotype_choice = (start_hap + locus_segments) % 2

    # Use `where` to select alleles from the chosen parental haplotype for each locus.
    # This is done for both geno and IBD arrays in parallel.
    gamete_geno = jnp.where(
        haplotype_choice == 0, parental_geno[0], parental_geno[1]
    )
    gamete_ibd = jnp.where(haplotype_choice == 0, parental_ibd[0], parental_ibd[1])

    return gamete_geno, gamete_ibd


@partial(jax.jit, static_argnames=("n_chr", "max_crossovers"))
def meiosis_for_one_cross(
    key: jax.random.PRNGKey,
    mother_geno: jnp.ndarray,  # Shape: (n_chr, ploidy, n_loci)
    father_geno: jnp.ndarray,  # Shape: (n_chr, ploidy, n_loci)
    mother_ibd: jnp.ndarray,
    father_ibd: jnp.ndarray,
    n_chr: int,
    gen_map: jnp.ndarray,  # Shape: (n_chr, n_loci)
    v_interference: float,
    max_crossovers: int,
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Creates a single diploid progeny's geno and IBD from two parents.
    This function is vmapped across chromosomes for parallel execution.
    """
    key_mother, key_father = jax.random.split(key)

    # Vectorize the gamete creation process across the chromosome axis.
    # in_axes specifies how to map the inputs over the vectorized axis (axis 0).
    vmapped_gamete_creator = vmap(
        _create_gamete,
        in_axes=(0, 0, 0, 0, None, None),  # Map over keys, geno, ibd, gen_map
        out_axes=(0, 0),
    )

    # Create gametes from the mother for all chromosomes in parallel.
    mother_gamete_geno, mother_gamete_ibd = vmapped_gamete_creator(
        jax.random.split(key_mother, n_chr),
        mother_geno,
        mother_ibd,
        gen_map,
        v_interference,
        max_crossovers,
    )

    # Create gametes from the father for all chromosomes in parallel.
    father_gamete_geno, father_gamete_ibd = vmapped_gamete_creator(
        jax.random.split(key_father, n_chr),
        father_geno,
        father_ibd,
        gen_map,
        v_interference,
        max_crossovers,
    )

    # Stack the two gametes to form the new diploid genotype and IBD.
    # The new ploidy axis is axis=1.
    progeny_geno = jnp.stack([mother_gamete_geno, father_gamete_geno], axis=1)
    progeny_ibd = jnp.stack([mother_gamete_ibd, father_gamete_ibd], axis=1)

    return progeny_geno, progeny_ibd


@partial(jax.jit, static_argnames=("config",))
def produce_offspring(
    key: jax.random.PRNGKey,
    state: SimState,
    sp: SimParam,
    config: SimConfig,
    mother_indices: jnp.ndarray, # (n_crosses,)
    father_indices: jnp.ndarray, # (n_crosses,)
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Top-level kernel to produce a cohort of offspring from selected parents.

    This function is designed to be the main entry point for meiosis in your
    simulation step. It is JIT-compatible and vmapped across crosses.
    """
    # Select the genotypes and IBD arrays for the chosen mothers and fathers.
    mothers_geno = state.geno[mother_indices]
    fathers_geno = state.geno[father_indices]
    mothers_ibd = state.ibd[mother_indices]
    fathers_ibd = state.ibd[father_indices]

    # Vectorize the single-cross meiosis function across the batch of parents.
    # This is the key to GPU acceleration: performing all crosses in parallel.
    vmapped_meiosis = vmap(
        meiosis_for_one_cross,
        in_axes=(0, 0, 0, 0, 0, None, None, None, None),
        out_axes=(0, 0),
    )

    n_crosses = mother_indices.shape[0]
    keys = jax.random.split(key, n_crosses)

    # Execute all crosses in parallel.
    offspring_geno, offspring_ibd = vmapped_meiosis(
        keys,
        mothers_geno,
        fathers_geno,
        mothers_ibd,
        fathers_ibd,
        config.n_chr,
        sp.gen_map,
        sp.recomb_params[0],  # Assuming v_interference is the first param
        20,  # max_crossovers, can be moved to config if needed
    )

    return offspring_geno, offspring_ibd


def make_crosses_robust(
    key: jax.random.PRNGKey,
    selected_indices: jnp.ndarray, # Indices of the n_select parents
    n_offspring: int,
):
    """
    Creates random mating pairs from a list of selected parents, assuming all 
    individuals are sexless.

    This function is robust to an empty list of selected parents.
    """
    # 1. Handle the case where no parents are selected to avoid JIT errors.
    has_parents = selected_indices.shape[0] > 0
    
    # 2. Randomly sample `n_offspring` mothers and fathers from the selected 
    #    individuals with replacement.
    key_mothers, key_fathers = jax.random.split(key)

    # If no parents are available, we'll get invalid indices (-1),
    # which can be handled downstream. The `jnp.where` prevents a JIT error.
    mothers = jax.random.choice(
        key_mothers,
        jnp.where(has_parents, selected_indices, -1),
        shape=(n_offspring,),
        replace=True
    )
    fathers = jax.random.choice(
        key_fathers,
        jnp.where(has_parents, selected_indices, -1),
        shape=(n_offspring,),
        replace=True
    )
    
    # Both `mothers` and `fathers` are now guaranteed to have shape (n_offspring,)
    return mothers, fathers


In [4]:
#| hide
import nbdev; nbdev.nbdev_export()